In [1]:
import pandas as pd
import os

In [2]:
years = ['2016', '2017', '2018', '2019']

In [7]:
first_run = True

for year in years:
    for fp in os.listdir(f'processed/{year}'):
        df = pd.read_csv(f'processed/{year}/{fp}')
        
        if not first_run:
            prev = pd.concat([prev, df], ignore_index=True)
        else:
            prev = df
            first_run = False

In [10]:
prev.drop(['Attend'], axis=1, errors='ignore')

,Team A Kills,Team B Kills,Team A Errors,Team B Errors,Team A Total Attacks,Team B Total Attacks,Team A Hit Pct,Team B Hit Pct,Team A Assists,Team B Assists,...,Assists,Aces,SErr,Digs,RErr,Block Solos,Block Assists,BErr,PTS,BHE
0,171.32,235.80,77.26,102.40,503.21,718.07,0.42,0.69,187.94,255.08,...,55,5,12,57,7,3,18,0,76.0,0
1,171.32,258.33,77.26,107.50,503.21,703.22,0.42,0.41,187.94,258.35,...,22,5,9,40,5,3,4,0,39.0,0
2,171.32,235.85,77.26,107.25,503.21,663.45,0.42,0.42,187.94,275.25,...,51,9,8,62,7,1,14,0,70.0,1
3,171.32,246.83,77.26,112.94,503.21,732.56,0.42,0.51,187.94,296.64,...,48,7,12,66,6,2,15,2,67.5,3
4,171.32,231.11,77.26,90.28,503.21,624.94,0.42,0.54,187.94,242.94,...,28,1,5,23,9,1,4,0,34.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38362,218.17,205.95,100.06,103.25,606.83,585.35,0.22,0.11,215.29,241.31,...,46,3,3,53,7,1,17,1,62.5,3
38363,218.17,245.53,100.06,109.12,606.83,697.18,0.22,0.15,215.29,276.86,...,51,3,13,68,2,1,13,1,65.5,2
38364,218.17,286.53,100.06,114.18,606.83,753.65,0.22,0.18,215.29,325.43,...,40,3,9,45,12,1,8,0,49.0,3
38365,218.17,254.38,100.06,105.69,606.83,653.88,0.22,0.20,215.29,293.08,...,54,6,7,75,7,6,10,1,79.0,4


In [11]:
prev.to_csv('merged-data.csv', index=False)

## Model for Merged Data - Error Predictions 

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [19]:
file_path = '/Users/ankitakhatri/Downloads/merged-data.csv'
data = pd.read_csv(file_path)
data = data.sample(n=len(data))
data.head()

,Team A Kills,Team B Kills,Team A Errors,Team B Errors,Team A Total Attacks,Team B Total Attacks,Team A Hit Pct,Team B Hit Pct,Team A Assists,Team B Assists,...,Aces,SErr,Digs,RErr,Block Solos,Block Assists,BErr,PTS,BHE,Attend
3485,280.06,242.16,126.76,105.42,834.88,651.63,0.38,0.56,271.56,306.14,...,3,5,67,2,2,8,3,64.0,0,NaN
30902,239.29,313.40,107.82,125.27,697.29,832.47,0.18,0.23,284.00,311.07,...,1,6,39,5,2,10,3,48.0,1,0.0
28630,220.06,151.23,113.65,80.18,657.82,444.68,0.05,0.10,203.00,169.72,...,10,9,34,14/,2,0,0,58.0,0,0.0
24291,232.88,232.47,98.88,85.42,702.12,600.95,0.20,-0.01,311.50,276.60,...,5,17,70,4,3,12,3,73.0,0,0.0
31328,253.82,205.85,104.71,84.45,776.29,608.25,0.22,0.14,267.27,242.88,...,6,9,104,2,1,19,1,74.5,0,0.0


In [ ]:
error_columns= ['SErr', 'RErr', 'BErr']

# average error for Team A and Team B
data['Avg Error'] = data[error_columns].mean(axis=1)

# Filter the dataset to include only relevant columns
team_columns = [col for col in data.columns if 'team a' in col.lower() or 'team b' in col.lower()]
teams_data = data[team_columns]
teams_data.head()

In [ ]:
# features (X) and targets (y)
X = teams_data
y = data['Avg Error']

# training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=60)

# train the model for A
model_team = RandomForestRegressor(n_estimators=100, random_state=42)
model_team.fit(X_train, y_train)

# predictions
y_pred = model_team.predict(X_test)

# mse
mse_team = mean_squared_error(y_test, y_pred)
